In [56]:
import pandas as pd

df = pd.read_excel('C:\\Users\\Biswajit Palit\\Empirical-Project\\clustering.xlsx')

In [57]:
df.head()

,Country,Country Code,Arable land,Crop production index,Exports of goods and services (% of GDP),External balance on goods and services (% of GDP),"Foreign direct investment, net inflows (% of GDP)",GDP per capita (constant 2015 US$),GDP per capita growth (annual %),"GNI per capita, PPP (current international $)",Gross domestic savings (% of GDP),Imports of goods and services (annual % growth),"Industry (including construction), value added (% of GDP)","International tourism, receipts (% of total exports)","Labor force participation rate, total (% of total population ages 15+) (modeled ILO estimate)",Net ODA received (% of GNI),"Population ages 15-64, total",Population growth (annual %),Rural population (% of total population),"Unemployment, youth total (% of total labor force ages 15-24) (national estimate)"
0,Afghanistan,AFG,11.774476,60.008333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.782333,NaN,8.792589e+06,3.910796,78.149667,NaN
1,Africa Eastern and Southern,AFE,6.119845,NaN,25.794606,-1.360998,1.576916,1256.887169,1.064457,1941.410883,13.502477,9.305430,26.879565,10.825900,71.616878,3.200903,1.958771e+08,2.593081,71.943787,NaN
2,Africa Western and Central,AFW,9.568818,NaN,NaN,NaN,0.909569,1181.812110,0.521767,NaN,NaN,NaN,32.819820,4.423629,65.486825,NaN,1.321472e+08,2.707933,65.822829,NaN
3,Albania,ALB,21.070560,50.556667,13.813751,-22.461537,2.500649,1662.045681,7.366358,3196.666667,-2.143339,18.812549,17.352575,42.328113,61.815000,9.265130,1.946905e+06,-0.627448,59.678333,NaN
4,Algeria,DZA,3.196962,35.623333,29.943287,6.549579,0.580733,2986.132952,1.861472,7396.666667,32.548581,1.549399,48.785009,NaN,44.460167,0.594805,1.760393e+07,1.612303,42.055833,NaN


In [58]:
df.columns

Index(['Country', 'Country Code', 'Arable land', 'Crop production index',
       'Exports of goods and services (% of GDP)',
       'External balance on goods and services (% of GDP)',
       'Foreign direct investment, net inflows (% of GDP)',
       'GDP per capita (constant 2015 US$)',
       'GDP per capita growth (annual %)',
       'GNI per capita, PPP (current international $)',
       'Gross domestic savings (% of GDP)',
       'Imports of goods and services (annual % growth)',
       'Industry (including construction), value added (% of GDP)',
       'International tourism, receipts (% of total exports)',
       'Labor force participation rate, total (% of total population ages 15+) (modeled ILO estimate)',
       'Net ODA received (% of GNI)', 'Population ages 15-64, total',
       'Population growth (annual %)',
       'Rural population (% of total population)',
       'Unemployment, youth total (% of total labor force ages 15-24) (national estimate)'],
      dtype='object')

In [59]:
data = pd.read_excel('C:\\Users\\Biswajit Palit\\Empirical-Project\\data_HDI.xlsx')
data.head()


,Country or Area,ISO-3 Code,HDI
0,Afghanistan,AFG,0.290000
1,Åland Islands,ALA,NaN
2,Albania,ALB,0.626667
3,Algeria,DZA,0.598333
4,American Samoa,ASM,NaN


In [60]:
import pandas as pd

# Assuming your DataFrame is named df
# Rename 'ISO-3 Code' column to 'Country Code'
data.rename(columns={'ISO-3 Code': 'Country Code'}, inplace=True)

# Now df will have the column 'ISO-3 Code' renamed to 'Country Code'


In [61]:
merged_df = pd.merge(df, data, on=['Country Code'], how='inner')


In [65]:
merged_df = merged_df.drop('Country or Area', axis = 1)

In [63]:
# Calculate the percentage of missing data for each country
missing_percentage = merged_df.isnull().mean(axis=1) * 100

# Assign weights based on missing percentage
weights = 100 - missing_percentage

# Add the weights as a new column in the dataset
merged_df['Weight'] = weights

In [84]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Select the columns you want to standardize
cols_for_standardization = merged_df.columns[2:-1]  # Exclude non-numeric columns and 'Weight', 'Country', and 'Country Code'

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to your selected columns and then transform them
standardized_data = scaler.fit_transform(merged_df[cols_for_standardization])

# Create a dataframe from the standardized data
standardized_df = pd.DataFrame(standardized_data, columns=cols_for_standardization)

# Keep the non-standardized columns in another dataframe
non_standardized_df = merged_df[['Country', 'Country Code', 'Weight']]

# Merge the standardized and non-standardized dataframes
merged_df = pd.concat([non_standardized_df, standardized_df], axis=1)


In [66]:
from sklearn.impute import KNNImputer

# Define the columns to be used for imputation (exclude 'Weight', 'Country', and 'Country Code')
cols_for_imputation = merged_df.columns[2:-1]  # Adjust the range as needed

# Initialize the KNNImputer with the desired number of neighbors (e.g., 5)
knn_imputer = KNNImputer(n_neighbors=5)

# Perform KNN imputation on the selected columns
merged_df[cols_for_imputation] = knn_imputer.fit_transform(merged_df[cols_for_imputation])


In [85]:
merged_df

,Country,Country Code,Weight,Arable land,Crop production index,Exports of goods and services (% of GDP),External balance on goods and services (% of GDP),"Foreign direct investment, net inflows (% of GDP)",GDP per capita (constant 2015 US$),GDP per capita growth (annual %),"GNI per capita, PPP (current international $)",Gross domestic savings (% of GDP),Imports of goods and services (annual % growth),"Industry (including construction), value added (% of GDP)","International tourism, receipts (% of total exports)","Labor force participation rate, total (% of total population ages 15+) (modeled ILO estimate)",Net ODA received (% of GNI),"Population ages 15-64, total",Population growth (annual %),Rural population (% of total population),"Unemployment, youth total (% of total labor force ages 15-24) (national estimate)",HDI,Weight
0,Afghanistan,AFG,45.454545,-0.128832,-0.557563,-0.095408,0.505506,-0.238336,-0.571702,0.038473,-0.486558,0.176118,-0.629824,0.117388,-0.756927,-1.524492,-0.051795,-0.114091,1.712568,1.351498,0.431273,-2.287390,-1.928629
1,Albania,ALB,95.454545,0.572891,-0.731245,-0.986573,-0.613148,-0.230033,-0.630393,1.122296,-0.547607,-1.228303,2.320227,-0.785049,1.333480,0.040601,0.297117,-0.208912,-1.548790,0.600365,0.912031,0.105460,0.607299
2,Algeria,DZA,90.909091,-0.776313,-1.005657,-0.252192,0.884944,-0.258859,-0.555909,-0.189767,-0.227895,1.046906,-1.228538,2.297052,-0.289029,-1.766259,-0.841347,0.007956,0.060781,-0.116251,-0.441155,-0.095918,0.376760
3,American Samoa,ASM,31.818182,-0.742744,1.860335,-0.374646,-1.555204,2.197366,0.932213,-0.033143,-0.186967,-0.082684,0.100693,-1.011795,1.508963,-0.098792,0.685751,-0.235440,0.191282,-1.299692,-0.358622,0.008325,-2.620246
4,Andorra,AND,45.454545,-0.857030,1.960483,0.143211,-0.919612,-0.152997,1.027101,0.124683,0.353804,-0.082684,0.100693,-1.031051,1.508963,-0.098792,-0.543321,-0.235226,-0.143971,-1.543512,0.287962,0.008325,-1.928629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,Virgin Islands (U.S.),VIR,36.363636,-0.442508,-0.024719,-0.127296,-1.553132,-0.149218,-0.202817,-0.042694,-0.104628,-0.082684,0.100693,-1.005197,1.952039,-0.007586,0.691232,-0.234917,-0.951020,-1.483186,0.088549,0.067554,-2.389707
212,West Bank and Gaza,PSE,95.454545,-0.093173,0.714289,-0.806598,-2.300858,-0.206812,-0.588980,-0.037916,-0.562705,-2.305809,-0.177356,-0.058700,0.513470,-2.204436,0.855292,-0.217237,1.429259,-0.656300,-0.270048,0.048601,0.607299
213,"Yemen, Rep.",YEM,95.454545,-0.790239,-0.552326,0.077209,0.310076,-0.297813,-0.605418,-0.125289,-0.572728,-0.238912,0.502398,1.582464,-0.498090,-1.683559,-0.428334,-0.122566,1.063291,1.223316,0.085167,-1.541105,0.607299
214,Zambia,ZMB,90.909091,-0.728851,-0.904958,-0.383696,0.176086,-0.205273,-0.678804,-0.423171,-0.673589,0.061026,5.143674,0.344572,-0.390339,-0.247826,2.204351,-0.172909,0.753941,0.778178,0.451746,-1.434493,0.376760


In [125]:
from scipy.cluster.hierarchy import linkage, fcluster

# Select the columns for clustering (excluding non-numeric columns)
cols_for_clustering = merged_df.columns[2:-1]  # Exclude non-numeric columns and the 'Weight' column

# Perform hierarchical clustering with weighted data
linkage_matrix = linkage(merged_df[cols_for_clustering], metric='mahalanobis')

# Define a threshold for identifying outliers
outlier_threshold = 10  # You can adjust this threshold as needed

# Assign clusters based on the threshold
clusters = fcluster(linkage_matrix, t = outlier_threshold, criterion='distance')

# Add the cluster assignments as a new column in the dataset
merged_df['Cluster'] = clusters

# Now, 'df' contains the cluster assignments for each country, taking into account the weights


LinAlgError: Singular matrix

In [121]:
help(fcluster)

Help on function fcluster in module scipy.cluster.hierarchy:

fcluster(Z, t, criterion='inconsistent', depth=2, R=None, monocrit=None)
    Form flat clusters from the hierarchical clustering defined by
    the given linkage matrix.
    
    Parameters
    ----------
    Z : ndarray
        The hierarchical clustering encoded with the matrix returned
        by the `linkage` function.
    t : scalar
        For criteria 'inconsistent', 'distance' or 'monocrit',
         this is the threshold to apply when forming flat clusters.
        For 'maxclust' or 'maxclust_monocrit' criteria,
         this would be max number of clusters requested.
    criterion : str, optional
        The criterion to use in forming flat clusters. This can
        be any of the following values:
    
          ``inconsistent`` :
              If a cluster node and all its
              descendants have an inconsistent value less than or equal
              to `t`, then all its leaf descendants belong to the
    

In [122]:
from collections import Counter

# Count the number of countries in each cluster
cluster_counts = Counter(merged_df['Cluster'])

# Identify clusters with fewer than 4 countries
small_clusters = [cluster for cluster, count in cluster_counts.items() if count < 4]

# Print the 'Country Code' values of rows in small clusters before dropping
for cluster in small_clusters:
    country_codes_to_drop = merged_df[merged_df['Cluster'] == cluster]['Country Code'].tolist()
    print(f"Country Codes in Cluster {cluster} to be dropped: {', '.join(country_codes_to_drop)}")




Country Codes in Cluster 7 to be dropped: AFG, AND, IMN
Country Codes in Cluster 23 to be dropped: ARM, ERI, GEO
Country Codes in Cluster 4 to be dropped: BMU, FRO
Country Codes in Cluster 21 to be dropped: BIH
Country Codes in Cluster 25 to be dropped: CHN
Country Codes in Cluster 5 to be dropped: DJI, PLW
Country Codes in Cluster 16 to be dropped: GNQ
Country Codes in Cluster 28 to be dropped: IND
Country Codes in Cluster 9 to be dropped: LIE
Country Codes in Cluster 6 to be dropped: STP, TUV
Country Codes in Cluster 14 to be dropped: KNA
Country Codes in Cluster 18 to be dropped: TLS
Country Codes in Cluster 15 to be dropped: ARE


In [123]:
merged_df

,Country,Country Code,Weight,Arable land,Crop production index,Exports of goods and services (% of GDP),External balance on goods and services (% of GDP),"Foreign direct investment, net inflows (% of GDP)",GDP per capita (constant 2015 US$),GDP per capita growth (annual %),"GNI per capita, PPP (current international $)",Gross domestic savings (% of GDP),Imports of goods and services (annual % growth),"Industry (including construction), value added (% of GDP)","International tourism, receipts (% of total exports)","Labor force participation rate, total (% of total population ages 15+) (modeled ILO estimate)",Net ODA received (% of GNI),"Population ages 15-64, total",Population growth (annual %),Rural population (% of total population),"Unemployment, youth total (% of total labor force ages 15-24) (national estimate)",HDI,Weight,Cluster
0,Afghanistan,AFG,45.454545,-0.128832,-0.557563,-0.095408,0.505506,-0.238336,-0.571702,0.038473,-0.486558,0.176118,-0.629824,0.117388,-0.756927,-1.524492,-0.051795,-0.114091,1.712568,1.351498,0.431273,-2.287390,-1.928629,7
1,Albania,ALB,95.454545,0.572891,-0.731245,-0.986573,-0.613148,-0.230033,-0.630393,1.122296,-0.547607,-1.228303,2.320227,-0.785049,1.333480,0.040601,0.297117,-0.208912,-1.548790,0.600365,0.912031,0.105460,0.607299,32
2,Algeria,DZA,90.909091,-0.776313,-1.005657,-0.252192,0.884944,-0.258859,-0.555909,-0.189767,-0.227895,1.046906,-1.228538,2.297052,-0.289029,-1.766259,-0.841347,0.007956,0.060781,-0.116251,-0.441155,-0.095918,0.376760,22
3,American Samoa,ASM,31.818182,-0.742744,1.860335,-0.374646,-1.555204,2.197366,0.932213,-0.033143,-0.186967,-0.082684,0.100693,-1.011795,1.508963,-0.098792,0.685751,-0.235440,0.191282,-1.299692,-0.358622,0.008325,-2.620246,3
4,Andorra,AND,45.454545,-0.857030,1.960483,0.143211,-0.919612,-0.152997,1.027101,0.124683,0.353804,-0.082684,0.100693,-1.031051,1.508963,-0.098792,-0.543321,-0.235226,-0.143971,-1.543512,0.287962,0.008325,-1.928629,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,Virgin Islands (U.S.),VIR,36.363636,-0.442508,-0.024719,-0.127296,-1.553132,-0.149218,-0.202817,-0.042694,-0.104628,-0.082684,0.100693,-1.005197,1.952039,-0.007586,0.691232,-0.234917,-0.951020,-1.483186,0.088549,0.067554,-2.389707,2
212,West Bank and Gaza,PSE,95.454545,-0.093173,0.714289,-0.806598,-2.300858,-0.206812,-0.588980,-0.037916,-0.562705,-2.305809,-0.177356,-0.058700,0.513470,-2.204436,0.855292,-0.217237,1.429259,-0.656300,-0.270048,0.048601,0.607299,30
213,"Yemen, Rep.",YEM,95.454545,-0.790239,-0.552326,0.077209,0.310076,-0.297813,-0.605418,-0.125289,-0.572728,-0.238912,0.502398,1.582464,-0.498090,-1.683559,-0.428334,-0.122566,1.063291,1.223316,0.085167,-1.541105,0.607299,30
214,Zambia,ZMB,90.909091,-0.728851,-0.904958,-0.383696,0.176086,-0.205273,-0.678804,-0.423171,-0.673589,0.061026,5.143674,0.344572,-0.390339,-0.247826,2.204351,-0.172909,0.753941,0.778178,0.451746,-1.434493,0.376760,24


In [80]:
# Drop countries belonging to small clusters
df = merged_df[~merged_df['Cluster'].isin(small_clusters)]

# Remove the 'Cluster' column, if no longer needed
df.drop(columns=['Cluster'], inplace=True)

C:\Users\Biswajit Palit\AppData\Local\Temp\ipykernel_17236\501094717.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['Cluster'], inplace=True)


In [81]:
df.head()

,Country,Country Code,Arable land,Crop production index,Exports of goods and services (% of GDP),External balance on goods and services (% of GDP),"Foreign direct investment, net inflows (% of GDP)",GDP per capita (constant 2015 US$),GDP per capita growth (annual %),"GNI per capita, PPP (current international $)",Gross domestic savings (% of GDP),Imports of goods and services (annual % growth),"Industry (including construction), value added (% of GDP)","International tourism, receipts (% of total exports)","Labor force participation rate, total (% of total population ages 15+) (modeled ILO estimate)",Net ODA received (% of GNI),"Population ages 15-64, total",Population growth (annual %),Rural population (% of total population),"Unemployment, youth total (% of total labor force ages 15-24) (national estimate)",HDI,Weight
